__[Open and try this file online (Colab)](https://colab.research.google.com/github/djekra/pandasklar/blob/master/jupyter/13_Analyse_Datatypes.ipynb)__

# Analyse Datatypes
* `analyse_datatypes`: Returns info about the datatypes and the mem_usage of the columns of a DataFrame 
* `analyse_values`: Returns statistical data for a DataFrame, a Series or an Index 
* `analyse_cols`: Describes the datatypes and the content of a DataFrame. Merged info from analyse_datatypes and analyse_values
* `change_datatype`: Converts the datatypes of a DataFrame or a Series. Automatically, if you want.
* `copy_datatype`: Copies the dtypes from one dataframe to another, matching the column names.

In [1]:
# blab init
try:
    import blab
except ImportError as e:
    !pip install blab
    import blab    
startup_notebook = blab.blab_startup()
%run $startup_notebook 

blab init
environment['in_colab']     = False
environment['dropbox_path'] = /home/me/Dropbox
environment['lib_path']     = /home/me/Dropbox/31_Projekte/01_Python/libs
Start Time: 13:20:13


In [2]:
import numpy      as np
import pandas     as pd 
import bpyth      as bpy

# pandasklar
try:
    import pandasklar as pak 
except ImportError as e:
    !pip install pandasklar
    import pandasklar as pak   
    
# verbose
pak.Config.set('VERBOSE', True)

# copy_on_write
pd.set_option("mode.copy_on_write", True)

VERBOSE = True
--> setting verbose=True as default for all pandasklar functions



In [3]:
# Generate random data
anz = 10000
a = pak.random_series( anz, 'int',   min=-500, max=100 )
b = pak.random_series( anz, 'int',   min=-127, max=127, p_dup=0 ) # keine Dups erlaubt
c = a + b + 0.0001
a = a % 10 * 10
v = pak.random_series( anz, 'name',                  p_nan=0)
w = v.str[:1]
s = pak.random_series( anz, 'string',                p_nan=0)
t = pak.random_series( anz, 'string',                p_nan=0.1)
m = pak.random_series( anz, 'int',   min=0, max=127, p_nan=0.1 )
n = pak.random_series( anz, 'float', decimals=4,     p_nan=0.2 ) #* 70000
o = pak.random_series( anz, 'choice', choice=['Bremen','Bremerhaven'], p_nan=0.3,   p_dup=0     )
p = pak.random_series( anz, 'list',                                    p_nan=0.1,   p_dup=0.5   )
q = pak.random_series( anz, 'time',                                    p_nan=0.1,   p_dup=0.5   )
z = pak.random_series( anz, 'mix',                                     p_nan=0.01,  p_dup=0     )

df = pak.dataframe( [a, b, c, v, w, s, t, m, n, o, p, q, z], verbose=False)
df.columns = ['int_grob','int_fein','float_summe','first_name','Letter1','string_nonan','string_nan','int_nan','float_nan','City','List','time','Mix']
df.float_summe = df.float_summe.astype('float')
pak.sample(df,10)

,int_grob,int_fein,float_summe,first_name,Letter1,string_nonan,string_nan,int_nan,float_nan,City,List,time,Mix
0,80,-117,-608.9999,Hanna,H,Fp8q,ÄZhHB,0,NaN,Bremen,NaN,1997-10-02 03:42:11.419243109,{0}
11,60,19,-454.9999,Elli,E,lC7w,<NA>,92,0.3412,Bremen,"[Willy, Mario]",NaT,Finn
24,0,91,-158.9999,Tom,T,ätöäGcv,ÖäxGdLg,36,NaN,Bremerhaven,"[Lena, Tom, Tanja, Vincent, Chiara, Aileen, Zoe]",2003-07-31 05:59:23.142410338,Bremerhaven
316,0,-119,-508.9999,Zoe,Z,PiEYX,s6303,63,NaN,Bremerhaven,"[Barbara, Tom, Julian, Natalie, Luca, Tom, Syl...",1934-04-07 21:59:37.654736525,1967-02-13 21:48:13.407160861
409,0,-46,-225.9999,Adolf,A,Öaäu,r3aGÄ3,<NA>,0.7222,NaN,"[Luisa, Ben]",1954-05-27 00:43:23.401311607,2020-06-16 20:44:52.107272351
2178,90,56,-194.9999,Anna,A,K9Ne,üü2hHts,114,0.6313,Bremen,"[Nadine, Tom, Swen, Sina, Tanja, Diana, Annett...",1949-08-09 00:40:01.256491040,Bremen
3817,80,-72,-143.9999,Margarete,M,ÜRckao,CHÄXhL,19,0.9999,Bremen,"[Ina, Tanja]",NaT,1998-03-26 07:50:25.187936316
7422,90,-10,-340.9999,Nico,N,ShhI,fBuDT,110,0.0000,Bremerhaven,"[Torben, Patrick, Anna, Elly, Raphael, Lucy, V...",NaT,JPJ6woQWJEECa
7992,80,41,-160.9999,Josephine,J,tNÖzn,00mDV,71,0.2969,NaN,"[Anja, Anja, Josef, Albert, Tanja, Albert, Tan...",NaT,-45698
9999,0,-73,-272.9999,Chiara,C,aDe2St,7HzÖ9u,121,0.5443,NaN,NaN,2019-04-03 13:53:02.172212312,"[Mika, Julian, Melina, Tom, Marina, Annika, To..."


## analyse_datatypes()

In [4]:
?pak.analyse_datatypes

Signature: pak.analyse_datatypes(df, with_index=True)
Docstring: Returns info about the datatypes and the mem_usage of the columns of a DataFrame.  
File:      ~/Dropbox/31_Projekte/01_Python/git/pandasklar/src/pandasklar/analyse.py
Type:      function

In [5]:
a = pak.analyse_datatypes(df)
a

,col_name,datatype_instance,datatype,datatype_short,is_numeric,is_string,is_datetime,is_hashable,nan_allowed,mem_usage
col_no,,,,,,,,,,
0,__index__,int64,np.int64,int64,True,False,False,True,False,80.0 B
1,int_grob,int64,np.int64,int64,True,False,False,True,False,80.0 B
2,int_fein,int64,np.int64,int64,True,False,False,True,False,80.0 B
3,float_summe,float64,np.float64,float64,True,False,False,True,True,80.0 B
4,first_name,str,pd.string,string,False,True,False,True,True,621.0 B
5,Letter1,str,pd.string,string,False,True,False,True,True,580.0 B
6,string_nonan,str,pd.string,string,False,True,False,True,True,717.0 B
7,string_nan,str,pd.string,string,False,True,False,True,True,812.0 B
8,int_nan,int64,pd.Int64,Int64,True,False,False,True,True,90.0 B


## analyse_values()

In [6]:
?pak.analyse_values

Signature:
pak.analyse_values(
    data,
    as_list=False,
    as_dict=False,
    sort=False,
    with_index=True,
    nanless_ints=False,
)
Docstring: Returns statistical data for a DataFrame, a Series or an Index     
File:      ~/Dropbox/31_Projekte/01_Python/git/pandasklar/src/pandasklar/analyse.py
Type:      function

In [7]:
pak.analyse_values(df)

,col_name,ntypes,nunique,nnan,ndups,n,vmin,vmean,vmedian,vmax,vsum,datatype_suggest,datatype_identified
col_no,,,,,,,,,,,,,
0,__index__,1,10000,0,0,10000,0.0,4999.50,4999.5,9999.0,49995000.00,np.int16,int
1,int_grob,1,10,0,9990,10000,0.0,45.09,50.0,90.0,450950.00,pd.Int8,int
2,int_fein,1,255,0,9745,10000,-127.0,0.20,0.0,127.0,1981.00,pd.Int8,int
3,float_summe,1,828,0,9172,10000,-624.0,-200.48,-203.0,225.0,-2004793.00,np.float32,float
4,first_name,1,435,0,9565,10000,Adolf,NaN,NaN,Zoe,NaN,,string
5,Letter1,1,24,0,9976,10000,A,NaN,NaN,Z,NaN,,string
6,string_nonan,1,10000,0,0,10000,002NW,NaN,NaN,üüm5a,NaN,,string
7,string_nan,1,8973,1027,0,10000,00mDV,NaN,NaN,üü2hHts,NaN,,string
8,int_nan,1,128,1005,8867,10000,0.0,63.37,63.0,127.0,570051.00,pd.Int8,int


## analyse_cols()

In [8]:
?pak.analyse_cols

Signature: pak.analyse_cols(df, sort=False, with_index=True)
Docstring:
Describes the datatypes and the content of a DataFrame.
Merged info from analyse_datatypes and analyse_values.
File:      ~/Dropbox/31_Projekte/01_Python/git/pandasklar/src/pandasklar/analyse.py
Type:      function

In [27]:
pak.analyse_cols(df)

,col_name,datatype_instance,datatype,datatype_short,datatype_suggest,datatype_identified,is_numeric,is_string,is_datetime,is_hashable,...,ntypes,nunique,nnan,ndups,n,vmin,vmean,vmedian,vmax,vsum
0,__index__,int64,np.int64,int64,np.int16,int,True,False,False,True,...,1,10000,0,0,10000,0.0,4999.50,4999.5,9999.0,49995000.00
1,int_grob,int64,np.int64,int64,pd.Int8,int,True,False,False,True,...,1,10,0,9990,10000,0.0,45.09,50.0,90.0,450950.00
2,int_fein,int64,np.int64,int64,pd.Int8,int,True,False,False,True,...,1,255,0,9745,10000,-127.0,0.20,0.0,127.0,1981.00
3,float_summe,float64,np.float64,float64,np.float32,float,True,False,False,True,...,1,828,0,9172,10000,-624.0,-200.48,-203.0,225.0,-2004793.00
4,first_name,str,pd.string,string,,string,False,True,False,True,...,1,435,0,9565,10000,Adolf,NaN,NaN,Zoe,NaN
5,Letter1,str,pd.string,string,,string,False,True,False,True,...,1,24,0,9976,10000,A,NaN,NaN,Z,NaN
6,string_nonan,str,pd.string,string,,string,False,True,False,True,...,1,10000,0,0,10000,002NW,NaN,NaN,üüm5a,NaN
7,string_nan,str,pd.string,string,,string,False,True,False,True,...,1,8973,1027,0,10000,00mDV,NaN,NaN,üü2hHts,NaN
8,int_nan,int64,pd.Int64,Int64,pd.Int8,int,True,False,False,True,...,1,128,1005,8867,10000,0.0,63.37,63.0,127.0,570051.00
9,float_nan,float64,np.float64,float64,np.float32,float,True,False,False,True,...,1,8020,1980,0,10000,0.0,0.50,0.5,1.0,4005.85


## change_datatype()

In [10]:
?pak.change_datatype

Signature:
pak.change_datatype(
    data,
    search=None,
    verbose=None,
    msg='',
    category_maxsize=-1,
    nanless_ints=False,
)
Docstring:
Converts the datatypes of a DataFrame or a Series.
If used with a Series:    
Similar behavior as pandas astype. But it also accepts
sloppy class names like type_info knows.
If no target datatype is specified, it will be selected automatically.
If used with a DataFrame:
Converts all datatypes automatically.                      

* category_maxsize: How big can a category get to be suggested as datatype_suggest?
* nanless_ints: Are numpy's integer classes (that don't know NaN) suggested as datatype_suggest?    
File:      ~/Dropbox/31_Projekte/01_Python/git/pandasklar/src/pandasklar/analyse.py
Type:      function

In [11]:
# example Series, before
pak.analyse_datatypes(df.int_fein)

,col_name,datatype_instance,datatype,datatype_short,is_numeric,is_string,is_datetime,is_hashable,nan_allowed,mem_usage
0,int_fein,int64,np.int64,int64,True,False,False,True,False,78.1 KB


In [12]:
# example Series, after 
a = pak.change_datatype(df.int_fein)
pak.analyse_datatypes(a)

int_fein             --> pd.Int8   


,col_name,datatype_instance,datatype,datatype_short,is_numeric,is_string,is_datetime,is_hashable,nan_allowed,mem_usage
0,int_fein,int8,pd.Int8,Int8,True,False,False,True,True,19.5 KB


In [13]:
# whole DataFrame
df2 = pak.change_datatype(df)
pak.analyse_cols(df2)

change_datatype 
int_grob             --> pd.Int8   
int_fein             --> pd.Int8   
float_summe          --> np.float32
int_nan              --> pd.Int8   
float_nan            --> np.float32
City                 --> pd.string 
change_datatype before: 5.1 MB after: 4.9 MB



,col_name,datatype_instance,datatype,datatype_short,datatype_suggest,is_numeric,is_string,is_datetime,is_hashable,nan_allowed,...,nunique,nnan,ndups,n,vmin,vmean,vmedian,vmax,vsum,datatype_identified
0,__index__,int64,np.int64,int64,np.int16,True,False,False,True,False,...,10000,0,0,10000,0.0,4999.50,4999.5,9999.0,49995000.00,int
1,int_grob,int8,pd.Int8,Int8,,True,False,False,True,True,...,10,0,9990,10000,0.0,45.09,50.0,90.0,450950.00,int
2,int_fein,int8,pd.Int8,Int8,,True,False,False,True,True,...,255,0,9745,10000,-127.0,0.20,0.0,127.0,1981.00,int
3,float_summe,float32,np.float32,float32,,True,False,False,True,True,...,828,0,9172,10000,-624.0,-200.48,-203.0,225.0,-2004793.38,float32
4,first_name,str,pd.string,string,,False,True,False,True,True,...,435,0,9565,10000,Adolf,NaN,NaN,Zoe,NaN,string
5,Letter1,str,pd.string,string,,False,True,False,True,True,...,24,0,9976,10000,A,NaN,NaN,Z,NaN,string
6,string_nonan,str,pd.string,string,,False,True,False,True,True,...,10000,0,0,10000,002NW,NaN,NaN,üüm5a,NaN,string
7,string_nan,str,pd.string,string,,False,True,False,True,True,...,8973,1027,0,10000,00mDV,NaN,NaN,üü2hHts,NaN,string
8,int_nan,int8,pd.Int8,Int8,,True,False,False,True,True,...,128,1005,8867,10000,0.0,63.37,63.0,127.0,570051.00,int
9,float_nan,float32,np.float32,float32,,True,False,False,True,True,...,8020,1980,0,10000,0.0,0.50,0.5,1.0,4005.85,float32


## copy_datatype()

In [14]:
?pak.copy_datatype

Signature: pak.copy_datatype(data_to, data_from)
Docstring:
Copies the dtypes from data_from to data_to. 
Usable for Series and DataFrames.
When applied on a DataFrame, it's applied to all column names that match.
File:      ~/Dropbox/31_Projekte/01_Python/git/pandasklar/src/pandasklar/pandas.py
Type:      function

In [15]:
# Target
df1 = df.copy()                      # Wrong datatypes
df1 = pak.drop_cols(df1,'int_fein')  # One column is missing

# Template
df2 = pak.change_datatype(df)        # Target datatypes
df2 = pak.drop_cols(df2,'City')      # One column is missing

change_datatype 
int_grob             --> pd.Int8   
int_fein             --> pd.Int8   
float_summe          --> np.float32
int_nan              --> pd.Int8   
float_nan            --> np.float32
City                 --> pd.string 
change_datatype before: 5.1 MB after: 4.9 MB



In [16]:
result = pak.copy_datatype(df1, df2)

# Result: All columns except 'City' were treated
pak.analyse_cols(result)

,col_name,datatype_instance,datatype,datatype_short,datatype_suggest,is_numeric,is_string,is_datetime,is_hashable,nan_allowed,...,nunique,nnan,ndups,n,vmin,vmean,vmedian,vmax,vsum,datatype_identified
0,__index__,int64,np.int64,int64,np.int16,True,False,False,True,False,...,10000,0,0,10000,0.0,4999.50,4999.5,9999.0,49995000.00,int
1,int_grob,int8,pd.Int8,Int8,,True,False,False,True,True,...,10,0,9990,10000,0.0,45.09,50.0,90.0,450950.00,int
2,float_summe,float32,np.float32,float32,,True,False,False,True,True,...,828,0,9172,10000,-624.0,-200.48,-203.0,225.0,-2004793.38,float32
3,first_name,str,pd.string,string,,False,True,False,True,True,...,435,0,9565,10000,Adolf,NaN,NaN,Zoe,NaN,string
4,Letter1,str,pd.string,string,,False,True,False,True,True,...,24,0,9976,10000,A,NaN,NaN,Z,NaN,string
5,string_nonan,str,pd.string,string,,False,True,False,True,True,...,10000,0,0,10000,002NW,NaN,NaN,üüm5a,NaN,string
6,string_nan,str,pd.string,string,,False,True,False,True,True,...,8973,1027,0,10000,00mDV,NaN,NaN,üü2hHts,NaN,string
7,int_nan,int8,pd.Int8,Int8,,True,False,False,True,True,...,128,1005,8867,10000,0.0,63.37,63.0,127.0,570051.00,int
8,float_nan,float32,np.float32,float32,,True,False,False,True,True,...,8020,1980,0,10000,0.0,0.50,0.5,1.0,4005.85,float32
9,City,str,object,object,pd.string,False,True,False,True,True,...,2,2981,7017,10000,Bremen,NaN,NaN,Bremerhaven,NaN,string


## type_info()

In [17]:
?pak.type_info

Init signature: pak.type_info(search)
Docstring:     
Provides information about pandas types and standardises them.
Is initialised with anything, e.g. with the name of a class, or with the class itself.
Or, even better, with a series.
Ex:   i = type_info('Int32')     
      i.info()            # returns all attributes, including for example:
      i.class_object      # the class object
      i.name              # the name of the Dtype
      i.name_instance     # type of the contents of the series
      i.instance1         # an example instance that is not NaN              
File:           ~/Dropbox/31_Projekte/01_Python/git/pandasklar/src/pandasklar/type_info.py
Type:           type
Subclasses:     

In [18]:
# Call with class name
i = pak.type_info('uint32')     
i.info()

{'instance1': None,
 'instance2': None,
 'name': 'np.uint32',
 'framework': 'np',
 'name_short': 'uint32',
 'name_long': 'np.uint32',
 'class_object': numpy.uint32,
 'is_hashable': True,
 'nan_allowed': False,
 'name_instance': '',
 'xmin': 0,
 'xmax': 4294967295}

In [19]:
# Call with class name
i = pak.type_info('intc')     
i.info()

{'instance1': None,
 'instance2': None,
 'name': 'np.intc',
 'framework': 'np',
 'name_short': 'intc',
 'name_long': 'np.intc',
 'class_object': numpy.int32,
 'is_hashable': True,
 'nan_allowed': False,
 'name_instance': '',
 'xmin': -2147483648,
 'xmax': 2147483647}

In [20]:
# Call with class name
i = pak.type_info('category')     
i.info()

{'instance1': None,
 'instance2': None,
 'name': 'pd.category',
 'framework': 'pd',
 'name_short': 'category',
 'name_long': 'pd.Categorical',
 'class_object': pandas.core.arrays.categorical.Categorical,
 'is_hashable': True,
 'nan_allowed': True,
 'name_instance': '',
 'xmin': None,
 'xmax': None}

In [21]:
# Call with sloppy class name
i = pak.type_info('pd.String')     
i = pak.type_info('str')  
i.info()

{'instance1': None,
 'instance2': None,
 'name': 'pd.string',
 'framework': 'pd',
 'name_short': 'string',
 'name_long': 'pd.StringDtype',
 'class_object': pandas.core.arrays.string_.StringDtype,
 'is_hashable': True,
 'nan_allowed': True,
 'name_instance': '',
 'xmin': None,
 'xmax': None}

In [22]:
# Call with class name
i = pak.type_info('StringDtype')     
i.info()

{'instance1': None,
 'instance2': None,
 'name': 'pd.string',
 'framework': 'pd',
 'name_short': 'string',
 'name_long': 'pd.StringDtype',
 'class_object': pandas.core.arrays.string_.StringDtype,
 'is_hashable': True,
 'nan_allowed': True,
 'name_instance': '',
 'xmin': None,
 'xmax': None}

In [23]:
# Call with class name
i = pak.type_info('float16')     
i.info()

{'instance1': None,
 'instance2': None,
 'name': 'np.float16',
 'framework': 'np',
 'name_short': 'float16',
 'name_long': 'np.float16',
 'class_object': numpy.float16,
 'is_hashable': True,
 'nan_allowed': True,
 'name_instance': '',
 'xmin': -65500.0,
 'xmax': 65500.0}

In [24]:
# Call with class 
i = pak.type_info(np.int16)     
i.info()

{'instance1': None,
 'instance2': None,
 'name': 'np.int16',
 'framework': 'np',
 'name_short': 'int16',
 'name_long': 'np.int16',
 'class_object': numpy.int16,
 'is_hashable': True,
 'nan_allowed': False,
 'name_instance': '',
 'xmin': -32768,
 'xmax': 32767}

In [25]:
# Call with class
i = pak.type_info(pd.Int64Dtype)     
i.info()

{'instance1': None,
 'instance2': None,
 'name': 'pd.Int64',
 'framework': 'pd',
 'name_short': 'Int64',
 'name_long': 'pd.Int64Dtype',
 'class_object': pandas.core.arrays.integer.Int64Dtype,
 'is_hashable': True,
 'nan_allowed': True,
 'name_instance': '',
 'xmin': -9223372036854775808,
 'xmax': 9223372036854775807}

In [26]:
# Call with Series
pak.type_info(df.List).info()

{'instance1': ['Nina',
  'Luka',
  'Anja',
  'Louisa',
  'Claudia',
  'Carl',
  'Paul',
  'Carsten',
  'Brigitte',
  'Volker'],
 'instance2': ['Leoni',
  'Lukas',
  'Marcus',
  'Maria',
  'Lina',
  'Anni',
  'Emily',
  'Paula',
  'Franz',
  'Andrea'],
 'name': 'object',
 'framework': '',
 'name_short': 'object',
 'name_long': 'object',
 'class_object': object,
 'is_hashable': False,
 'nan_allowed': True,
 'name_instance': 'list',
 'xmin': None,
 'xmax': None}